In [7]:
import numpy as np
import joblib as joblib
# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import os
import sys

from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow
from tensorflow import keras

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

model = keras.models.load_model("C:/Users/Jonathan Munoz/Projects/ToneTendency/AudioAnalysis/ModelData/audiomodel")
encoder = joblib.load("C:/Users/Jonathan Munoz/Projects/ToneTendency/AudioAnalysis/ModelData/encoder.save") 
scaler = joblib.load("C:/Users/Jonathan Munoz/Projects/ToneTendency/AudioAnalysis/ModelData/scaler.save") 
actorpath = "C:/Users/Jonathan Munoz/Projects/ToneData/Audio_Speech_Actors_01-24/Actor_01/"
audio_predict_order = ['angry', 'calm', 'digust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


def extract_features(data, sample_rate):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

def predict(path) :
    data, sample_rate = librosa.load(path)
    data, _ = librosa.effects.trim(data, top_db=20, frame_length=256, hop_length=64)
    X = []
    X.append(np.array(extract_features(data, sample_rate)))
    X = scaler.transform(X)
    x_pred = np.expand_dims(X, axis=2)
    x_pred = model.predict(x_pred)
    result = {}
    for x in range(8):
        result[audio_predict_order[x]]= x_pred[0,x]
    return result;

for file in os.listdir("C:/Users/Jonathan Munoz/Projects/ToneData/Audio_Speech_Actors_01-24/Actor_01/"):
    print(predict(actorpath+file))

{'angry': 0.008202264, 'calm': 0.00059687457, 'digust': 0.09205233, 'fear': 0.06863171, 'happy': 0.1489085, 'neutral': 0.16170587, 'sad': 0.49544334, 'surprise': 0.024459139}
{'angry': 0.00273965, 'calm': 1.7067315e-06, 'digust': 0.14680329, 'fear': 0.079801545, 'happy': 0.07094423, 'neutral': 0.070644446, 'sad': 0.6072096, 'surprise': 0.021855505}
{'angry': 0.0002688103, 'calm': 1.2985578e-05, 'digust': 0.009303836, 'fear': 0.120906584, 'happy': 0.070367984, 'neutral': 0.020947771, 'sad': 0.7344354, 'surprise': 0.0437567}
{'angry': 0.0011091965, 'calm': 8.141735e-10, 'digust': 0.013044189, 'fear': 0.15036412, 'happy': 0.08340388, 'neutral': 0.002498538, 'sad': 0.5345333, 'surprise': 0.21504684}
{'angry': 0.01511742, 'calm': 0.00029966809, 'digust': 0.041292444, 'fear': 0.09146278, 'happy': 0.33049226, 'neutral': 0.081383936, 'sad': 0.4051315, 'surprise': 0.03481999}
{'angry': 0.0027950618, 'calm': 6.191719e-05, 'digust': 0.07420496, 'fear': 0.04990975, 'happy': 0.10098782, 'neutral': 

KeyboardInterrupt: 